In [1]:
import copy
import numpy as np
import itertools


In [2]:
def split(p1, p2):
    [a, b, c, d] = sorted([*p1, *p2])
    
    if (a == b and b == c and c == d):
        return [(a, a)]
    elif (a == b and b == c):
        return [(a, a), (a+1, d)]
    elif (b == c and c == d):
        return [(a, b-1), (b, b)]
    elif (a == b) and (c == d):
        return [(a, c)]
    elif (a == b):
        return [(a, c), (c+1, d)]
    elif (b == c):
        return [(a, b-1), (b, b), (b+1, d)]
    elif (c == d):
        return [(a, b-1), (b+1, d)]
    else:
        return [(a, b-1), (b, c), (c+1, d)]

In [3]:
split((1, 9), (1, 4))

[(1, 4), (5, 9)]

In [4]:

class Rectangle:
    def intersection(self, other):
        a, b = self, other
        x1 = max(min(a.x1, a.x2), min(b.x1, b.x2))
        y1 = max(min(a.y1, a.y2), min(b.y1, b.y2))
        x2 = min(max(a.x1, a.x2), max(b.x1, b.x2))
        y2 = min(max(a.y1, a.y2), max(b.y1, b.y2))
        if x1 <= x2 and y1 <= y2:
            return type(self)(x1, x2, y1, y2)
    __and__ = intersection

    '''
    def difference(self, other):
        inter = self & other
        
        # we've ensured an intersection
        if not inter:
            yield self
            return
        
        x_split = split((self.x1, self.x2), (other.x1, other.x2))
        y_split = split((self.y1, self.y2), (other.y1, other.y2))
        
        print(x_split)
        print(y_split)
                                   
        for (x1, x2), (y1, y2) in itertools.product(x_split, y_split):
            rect = type(self)(x1, x2, y1, y2)
            
            if not rect.subset(inter) and rect.subset(self):
                yield rect
    '''
    
    def complement(self):
        xs = [(-np.inf, self.x1-1), (self.x1, self.x2), (self.x2+1, np.inf)]
        ys = [(-np.inf, self.y1-1), (self.y1, self.y2), (self.y2+1, np.inf)]
        
        for (x1, x2), (y1, y2) in itertools.product(xs, ys):
            rect = type(self)(x1, x2, y1, y2)
            
            if rect != self:
                yield rect
    
    def difference(self, other):
        inter = self & other
        
        # we've ensured an intersection
        if not inter:
            yield self
            return
                
        for c in other.complement():
            i = self & c
            if i:
                yield i

    __sub__ = difference

    def __init__(self, x1, x2, y1, y2):
        self.x1, self.x2, self.y1, self.y2 = x1, x2, y1, y2

    def __iter__(self):
        yield self.x1
        yield self.x2
        yield self.y1
        yield self.y2

    def __eq__(self, other):
        return isinstance(other, Rectangle) and tuple(self) == tuple(other)
    def __ne__(self, other):
        return not (self == other)

    def __repr__(self):
        return type(self).__name__ + repr(tuple(self))
    
    def volume(self):
        return (self.y2 - self.y1 + 1) * (self.x2 - self.x1 + 1)
    
    def subset(self, other):
        return self == self & other
    
    def union(self, other):
        # guaranteed no overlap
        inter = self & other
        
        if inter:
            yield inter
        
        yield from self.difference(other)
        yield from other.difference(self)
    __or__ = union
    
    def __copy__(self):
        return type(self)(self.x1, self.x2, self.y1, self.y2)

In [5]:
cubes = [
    Rectangle(-2, 5, 0, 3),
    Rectangle(-2, 1, 0, 3),
    Rectangle(-4, 4, 0, 6),
    Rectangle(-1, 2, -1, 3),
    Rectangle(-4, 4, 0, 6),
    Rectangle(-1, 2, -1, 0),
    Rectangle(-4, 4, 0, 6),
    Rectangle(-4, 4, 0, 6)
]

test_cases = itertools.combinations(cubes, 2) 

for (a, b) in test_cases:    
    # none of differences have any overlap
    for (c, d) in itertools.combinations(a.difference(b), 2):
        assert c & d is None
        assert d & c is None
        
    inter = a & b
    inter_volume = inter.volume() if inter else 0
    
    print(a, b)
    
    assert a.volume() - inter_volume == sum(c.volume() for c in a.difference(b))
    assert b.volume() - inter_volume == sum(c.volume() for c in b.difference(a))
    
    assert a.volume() + b.volume() - inter_volume == sum(c.volume() for c in a.union(b))
    assert a.volume() + b.volume() - inter_volume == sum(c.volume() for c in b.union(a))

Rectangle(-2, 5, 0, 3) Rectangle(-2, 1, 0, 3)
Rectangle(-2, 5, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-2, 5, 0, 3) Rectangle(-1, 2, -1, 3)
Rectangle(-2, 5, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-2, 5, 0, 3) Rectangle(-1, 2, -1, 0)
Rectangle(-2, 5, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-2, 5, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-2, 1, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-2, 1, 0, 3) Rectangle(-1, 2, -1, 3)
Rectangle(-2, 1, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-2, 1, 0, 3) Rectangle(-1, 2, -1, 0)
Rectangle(-2, 1, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-2, 1, 0, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-4, 4, 0, 6) Rectangle(-1, 2, -1, 3)
Rectangle(-4, 4, 0, 6) Rectangle(-4, 4, 0, 6)
Rectangle(-4, 4, 0, 6) Rectangle(-1, 2, -1, 0)
Rectangle(-4, 4, 0, 6) Rectangle(-4, 4, 0, 6)
Rectangle(-4, 4, 0, 6) Rectangle(-4, 4, 0, 6)
Rectangle(-1, 2, -1, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-1, 2, -1, 3) Rectangle(-1, 2, -1, 0)
Rectangle(-1, 2, -1, 3) Rectangle(-4, 4, 0, 6)
Rectangle(-1, 2, -1, 3) 

In [6]:
a = Rectangle(-2, 5, 0, 3)
b = Rectangle(-1, 2, -1, 3)

In [7]:
inter = a & b
inter_volume = inter.volume() if inter else 0

In [8]:
inter

Rectangle(-1, 2, 0, 3)

In [9]:
list(a.difference(b))

[Rectangle(-2, -2, 0, 3), Rectangle(3, 5, 0, 3)]

In [10]:
a.volume() - inter_volume == sum(c.volume() for c in a.difference(b))

True

In [11]:
split((-1, 3), (0, 3))

[(-1, -1), (1, 3)]